In [1]:
from logging import getLogger
from logging.config import dictConfig

from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAUROC
from yaml import safe_load

from streamvigil import ADBenchDataset
from streamvigil.detectors import RSRAE

In [2]:
# Logger
with open("./logging.yml", encoding="utf-8") as file:
        config = safe_load(file)
dictConfig(config)
logger = getLogger(__name__)

In [3]:
# Anomaly Detector
detector = RSRAE(
    encoder_dims=[500, 450, 400, 350, 300],
    decoder_dims=[250, 300, 350, 400, 450, 500],
    rsr_dim=250,
    batch_norm=True,
)

In [4]:
random_state = 42

# Load dataset
train_data = ADBenchDataset(
    "../data/9_census.npz",
    train=True,
    random_state=random_state,
)
test_data = ADBenchDataset(
    "../data/9_census.npz",
    train=False,
    random_state=random_state,
)

# DataLoader
train_loader = DataLoader(train_data, batch_size=512)
test_loader = DataLoader(test_data, batch_size=256)

In [5]:
# Training
epochs = 10
logger.info("Start training the model...")
for epoch in range(epochs):
    logger.info(f"Epoch: {epoch+1}")
    for batch, (X, _) in enumerate(train_loader):
        loss = detector.train(X)

        if batch % 100 == 0:
            logger.info(f"Loss: {loss.item():>7f}")
logger.info("Completed training the model!")

2024-04-24 22:16:13,208 [INFO] __main__: Start training the model...
2024-04-24 22:16:13,208 [INFO] __main__: Epoch: 1
2024-04-24 22:16:13,491 [INFO] __main__: Loss: 9005.844727
2024-04-24 22:16:15,421 [INFO] __main__: Loss: 1779.112427
2024-04-24 22:16:17,508 [INFO] __main__: Loss: 1461.656982
2024-04-24 22:16:19,576 [INFO] __main__: Loss: 1339.477539
2024-04-24 22:16:21,745 [INFO] __main__: Loss: 1360.775513
2024-04-24 22:16:21,918 [INFO] __main__: Epoch: 2
2024-04-24 22:16:21,939 [INFO] __main__: Loss: 1367.468994
2024-04-24 22:16:23,959 [INFO] __main__: Loss: 1316.431885
2024-04-24 22:16:25,914 [INFO] __main__: Loss: 1243.711792
2024-04-24 22:16:27,981 [INFO] __main__: Loss: 1176.703979
2024-04-24 22:16:29,967 [INFO] __main__: Loss: 1204.854126
2024-04-24 22:16:30,146 [INFO] __main__: Epoch: 3
2024-04-24 22:16:30,168 [INFO] __main__: Loss: 1223.594604
2024-04-24 22:16:32,155 [INFO] __main__: Loss: 1196.565796
2024-04-24 22:16:34,204 [INFO] __main__: Loss: 1131.874756
2024-04-24 22:

In [6]:
# Evaluation
metrics = BinaryAUROC()
logger.info("Start evaluating the model...")

for X, y in test_loader:
    scores = detector.predict(X)
    metrics.update(scores, y)

# Compute AUROC score
logger.info(f"AUROC Score: {metrics.compute()}")

logger.info("Completed the evaluation of the model!")

2024-04-24 22:17:35,575 [INFO] __main__: Start evaluating the model...
2024-04-24 22:17:36,952 [INFO] __main__: AUROC Score: 0.4031903755205685
2024-04-24 22:17:36,952 [INFO] __main__: Completed the evaluation of the model!
